# 东方财富网国内新闻数据抓取器
## 拂晓工作室
### 此程序将爬取某个板块所有最近的新闻并存进CSV！

# 参考资料
* [1] BeautifulSoup的例子：https://qiita.com/rusarusa/items/d7f014ba80d6fe7a3e07
* [2] BeautifulSoup的文档：https://www.cnblogs.com/zhaof/p/6930955.html
* [3] How to install BS4 aggressively: https://github.com/ipython/ipython/issues/10684
* [4] CSV文档：https://docs.python.org/2/library/csv.html
* [5] CSV使用实例：https://blog.csdn.net/waple_0820/article/details/70049953
* [6] 如何解决list重复元素的问题：https://stackoverflow.com/questions/7961363/removing-duplicates-in-lists

In [66]:
# 导入库
import urllib
import re
import pandas as pd
import pymysql
import os
import turicreate as tc
import csv
from bs4 import BeautifulSoup


In [13]:
# 新闻板块(同时也会加入热门新闻的数据)
china = 'cgnjj'  # 国内
international = 'cgjjj'  # 国际

# 定义基本常量
url = 'http://finance.eastmoney.com/news/' + china + '.html' # 主目录地址 更换不同主目录以爬取不同板块内容 (BAD)
csv_filepath = '../Datasets/Eastmoney/news/'  # 存储数据文件地址

In [65]:
# 创建一个二十五页的list
page_list = []
counter = 1

while counter <= 25:
    pageurl = 'http://finance.eastmoney.com/news/' + china
    if counter != 1:
        pageurl = pageurl + '_' + str(counter) + '.html'
        page_list.append(pageurl)
    else:
        pageurl = pageurl + '.html'
        page_list.append(pageurl)
    counter += 1
    
print('成功创建包含 ' + str(len(page_list)) + ' 个页面链接的目录！')

成功创建包含 25 个页面链接的目录！


In [69]:
# 创建一个包含20个class的list
counter = 0
class_list = []

while counter < 20:
    class_list.append('newsTr' + str(counter))
    counter += 1
    
print('成功创建包含 ' + str(len(class_list)) + ' 个class的目录！')
# print(class_list)  # Debug

成功创建包含 20 个class的目录！


In [80]:
# 初始化urllist
urllist = []
year = '2018'  # Separate parameter
counter = 1

# 创造urllist
print('-----------------获取所有文章超链接程序-----------------')
print('分析任务开始！')
for url in page_list:
    req = urllib.request.Request(url)
    response = urllib.request.urlopen(req)
    html = response.read()
    soup = BeautifulSoup(html, "lxml")
    for each_url in soup.find_all('a', href=True):
        if 'http://finance.eastmoney.com/news/' in each_url['href']:
            if year in each_url['href']:
                urllist.append(each_url['href'])   
    print('第' + str(counter) + '页分析完成！现在一共有' + str(len(urllist)) + '篇文章的数据！')
    counter += 1

urllist = list(set(urllist))
print('全部分析完成！正在查重...')
print('任务完成！共获得了' + str(len(urllist)) + '篇文章的数据！')
print('---------------------------------------------')

-----------------获取所有文章超链接程序-----------------
分析任务开始！


第1页分析完成！现在一共有41篇文章的数据！


第2页分析完成！现在一共有76篇文章的数据！


第3页分析完成！现在一共有111篇文章的数据！


第4页分析完成！现在一共有149篇文章的数据！


...

第13页分析完成！现在一共有483篇文章的数据！


第14页分析完成！现在一共有519篇文章的数据！


第15页分析完成！现在一共有554篇文章的数据！


第16页分析完成！现在一共有591篇文章的数据！


IncompleteRead: IncompleteRead(36134 bytes read, 47535 more expected)

In [11]:
# 抓取新闻URL函数
def collectNews(url):
    # 准备工作
    req = urllib.request.Request(url)
    response = urllib.request.urlopen(req)
    html = response.read()
    soup = BeautifulSoup(html, "lxml")
    
    # 获得文章的标题
    title = soup.find('h1').get_text()
    print(title)  # Debug
    
    # 获得文章的发表时间
    time = content = soup.find(class_="time").get_text()
    year = time[0:4]
    month = time[5:7]
    day = time[8:10]
    print(time,year,month,day)  # Debug
    
    # 获得文章的内容
    content = soup.find(id="ContentBody").get_text()
    index = re.search('责任编辑', content).span()[0]-1
    content = content[0:index].strip()
    print(content)  # Debug
    
    # 获得相关的股票单
    related_stock = soup.find_all(class_='keytip')
    related_stocks = []
    for each in related_stock:
        stock = each.get_text()
        related_stocks.append(stock)
        # print(each.get('href'))  #大概不需要URL
    print(related_stocks)
    
    # 将list处理成String 中间用&分割
    related_stocks_string = ''
    for stock in related_stocks:
        related_stocks.append('&')
        related_stocks.append(stock)
        
    # 写入csv
    writer.writerow([counter, year, month, day, title, contents, related_stocks_string])
    
    # 刷新计数
    counter += 1

In [ ]:
# Index Counter
counter = 0